In [1]:
import fcsugar
from fcsugar import Container, Node, node
from fcsugar.containers import DataFrameContainer
from fcsugar.library import *
from fcsugar.plotting import map_labels_to_colors
import pandas as pd
from bokeh.plotting import figure as figure, gridplot
from bokeh.colors.rgb import RGB
from bokeh.io import output_notebook, push_notebook, show
output_notebook()
from scipy import fftpack
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
from IPython.display import clear_output

Loading BokehJS ...

## Create a bokeh figure.
Initialize the scatter plot with a dummy datapoint, otherwise you cannot update it with real data from the pipeline.

In [2]:
f1 = figure(plot_width=500, plot_height=500)#, y_range=(-0.1, 1.1))
g1 = f1.scatter(np.arange(1), np.arange(1), color=['black'], alpha=0.7)
p = gridplot([[f1]])
show(p, notebook_handle=True)

## Function to update the plot when the pipeline finishes

In [3]:
def plot_data(container: DataFrameContainer):
    data = np.vstack(container.dataframe['lda_transform'].values)
    g1.data_source.data['x'] = data[:, 0]
    g1.data_source.data['y'] = data[:, 1]
    g1.data_source.data['fill_color'] = map_labels_to_colors(container.dataframe['FCLUSTER_LABELS'], 'tab10', output='bokeh')
    g1.data_source.data['line_color'] = map_labels_to_colors(container.dataframe['FCLUSTER_LABELS'], 'tab10', output='bokeh')
    
    push_notebook()

### Open a container

In [4]:
cont = DataFrameContainer.from_hdf5('./data.h5', key='data')

Textarea(value='', description='Status', layout=Layout(width='80%'))

## Connect the pipeline output to the above plot
By doing this the container is passed to `plot_data()` every time that a pipeline is completed

In [5]:
cont.connect(plot_data)

**Make sure `show_gui` is true in the config**

In [6]:
fcsugar.config.show_gui = True

## Define the pipeline & execute
### The pipeline will re-execute & update the plot when you make changes in the GUI

In [7]:
cont \
>> splice('_RAW_CURVE', 0, 2990)\
>> normalize('spliced')\
>> rfft('normalize')\
>> absval('fft')\
>> log('absval')\
>> splice('log', 0, 1000)\
>> LDA('spliced', labels_column='FCLUSTER_LABELS', n_components=2)
cont.execute_pipeline(clear=False) # Don't clear the pipeline so we can use the GUI to play around with the parameters

HTML(value='<b>splice</b>')

Text(value='_RAW_CURVE', description='data_column')

IntText(value=0, description='start')

IntText(value=2990, description='stop')

HTML(value='<b>normalize</b>')

...

HTML(value='<b>LDA</b>')

Text(value='spliced', description='data_column')

Text(value='FCLUSTER_LABELS', description='labels_column')

IntText(value=2, description='n_components')

/home/kushal/python-virtual-environments/mesmerize/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
